# Capstone Project - The Battle of Neighborhoods (Week 1)

 **_How to choose the best hotel in Paris?_**

## I. Business Problem Section

### Introduction

Being one of the world capitals of arts, culture, gastronomy and fashion, millions of travelers visit Paris each year to explore the city's cultural attractions, such as: The Eiffel Tower, Musée du Louvre, Cathédrale Notre-Dame de Paris, Avenue des Champs-Élysées, Disneyland, Palace of Versailles, Musée d'Orsay and so on. There are a lot of travel agencies that offer various deals on flights, hotel stays and rental cars. Also, there are people who prefer not to work with a travel agency and who want to plan the holiday on their own. 

If someone wants to travel there are many things to consider from choosing the right location, accomodation, flights, rental cars to attractions, restaurants, stores and other facilities. 

Therefore, the main idea is that in both cases, on their own and with a travel agency, it is necessary to have a list of recommendations and criteria of choosing the optimal one. So, a good idea would we to develop one aplication that incorporate a lot of machine learning techniques and leverage the Foursquare location data in order to cluster different cities neighborhoods, in our case Paris, to recommend venues and to support people who are looking for the right hotel to take the best decisions.

### Business Problem

In this scenario, the business problem I am trying to solve is: How could I provide support to different stakeholders (people or tourism agencies) in choosing the best accomodation? Where would I recommend that is the best place to stay?

To solve this business problem, we will use Foursquare location data and we will create machine learning models to cluster Paris neighborhoods in order to recommend profitable hotels based on different surrounding facilities such venues, restaurants, stores, attractions and so on.

Through these models the stakeholders will have a wide range of recommentations for accomodation, they will know all the facitilies to enjoy on vacation, will receive a wide range of options and, in this way, they will know exactly what hotel is the most suitable for them.

## II. Data Section

To perform this idea, it was used data from 2 different sources. Data about Paris hotels was taken from: https://www.accorhotels.com. It was collected information related to postal code, name of the hotel, address and it was integrated into a database which contain 40 observations about 40 hotels from Paris. For a better analysis it was selected data about hotels from different areas and with different facilities.

The second source used is Foursquare location data in order to explore and target recommended locations across different venues. Everything was arranged into a pandas dataframe for exploration, visualization and modeling.

The final database which combine Foursquare location data and Paris hotels data, will be used to develop our machine learning models and to cluster Paris neighborhoods in order to provide the best recommendations in choosing a hotel based on a wide range of surrounding facilities.